# Machine Learning in Chess to Study Patterns in Chess Games

## Import Libraries

In [1]:
import pandas as pd

## Load the Data

In [2]:
CSV_PATH = "/Users/isaac/Downloads/ChessDBs/lichess_db_standard_rated_2022-08.csv"
# Check whether the DataFrame has been loaded correctly.
df = pd.read_csv(CSV_PATH)
display(df.info())
display(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6260667 entries, 0 to 6260666
Data columns (total 13 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   UTCDate      object
 1   UTCTime      object
 2   Event        object
 3   TimeControl  object
 4   Result       object
 5   Termination  object
 6   ECO          object
 7   Opening      object
 8   White        object
 9   Black        object
 10  WhiteElo     int64 
 11  BlackElo     int64 
 12  Site         object
dtypes: int64(2), object(11)
memory usage: 620.9+ MB


None

,UTCDate,UTCTime,Event,TimeControl,Result,Termination,ECO,Opening,White,Black,WhiteElo,BlackElo,Site
0,2022.08.01,00:00:37,Rated Bullet game,60+0,1/2-1/2,Time forfeit,B06,Modern Defense,juanstereo,LUGZ,1923,1910,https://lichess.org/aAcMUViA
1,2022.08.01,00:00:37,Rated Bullet game,60+0,0-1,Time forfeit,B01,Scandinavian Defense: Main Line,PaulBeckwith,nathanng88,1781,1782,https://lichess.org/brBOPdLj
2,2022.08.01,00:00:37,Rated Bullet game,60+0,1-0,Normal,A00,Hungarian Opening: Slav Formation,DrMaimer,lowliehomie,2528,2355,https://lichess.org/pk0m4lj8
3,2022.08.01,00:00:37,Rated Bullet game,60+0,1-0,Time forfeit,B00,Rat Defense: Antal Defense,FlyingPig,sabinayallan,2010,1996,https://lichess.org/ZXitDcRa
4,2022.08.01,00:00:37,Rated Bullet game,60+0,0-1,Time forfeit,C47,Four Knights Game: Scotch Variation Accepted,Edbunker,mirza47,1805,1831,https://lichess.org/j50FcHUm
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6260662,2022.08.03,11:18:48,Rated Blitz game,300+3,1-0,Normal,C00,French Defense: Knight Variation,bparau,Mina100,1921,1941,https://lichess.org/bqbUnnON
6260663,2022.08.03,11:18:48,Rated Bullet game,120+0,1-0,Time forfeit,C50,"Italian Game: Giuoco Pianissimo, Italian Four ...",KOTBAPKOT,peterdavidson,1497,1576,https://lichess.org/bWscvrwe
6260664,2022.08.03,11:18:48,Rated Bullet game,60+0,0-1,Normal,C25,Vienna Game: Vienna Gambit,asia53059,naman_gg,1476,1408,https://lichess.org/6kfsAGi6
6260665,2022.08.03,11:18:48,Rated Blitz game,180+0,1-0,Normal,C01,"French Defense: Exchange Variation, Svenonius ...",XefeBLINK,Akmuslum,1497,1486,https://lichess.org/uYjenmdN


## Get Most Popular Openings

In [5]:
display(df["Opening"].value_counts())

Modern Defense                                                         161051
Queen's Pawn Game                                                      160099
Scandinavian Defense: Mieses-Kotroc Variation                          137813
Caro-Kann Defense                                                      122481
Philidor Defense                                                       112600
                                                                        ...  
Pterodactyl Defense: Sicilian, Siroccopteryx                                1
Scandinavian Defense: Modern Variation, Wing Gambit                         1
Queen's Gambit Accepted: Classical Defense, Russian Gambit                  1
Grünfeld Defense: Lundin Variation                                          1
Semi-Slav Defense: Meran Variation, Wade Variation, Kaidanov Gambit         1
Name: Opening, Length: 2819, dtype: int64

## Get Most Popular Opening Categories

In [4]:
display(df["ECO"].value_counts())

A00    382967
A40    346568
B01    330680
D00    275919
B00    270159
        ...  
E96         5
A79         5
D68         4
D69         4
D65         2
Name: ECO, Length: 496, dtype: int64